In [1]:
# ==============================================================================
# STAGE 1, BLOCK 1 (NEW METHOD): UPLOAD & UNZIP
# ==============================================================================
import os

# Make sure you've uploaded 'ML_Challenge_Data.zip' before running this
zip_file_path = 'ML_Challenge_Data.zip'

if os.path.exists(zip_file_path):
    print("⏳ Unzipping the data files...")
    # The '!' runs a shell command. '-q' makes the output quiet.
    !unzip -q {zip_file_path}
    # Move the contents up to the main /content/ directory
    !mv /content/ML_Challenge_Data/* /content/
    print("✅ Files unzipped successfully!")
    print("Your 'dataset' and 'src' folders are now ready.")
else:
    print(f"❌ Error: '{zip_file_path}' not found. Please upload it first.")

⏳ Unzipping the data files...
✅ Files unzipped successfully!
Your 'dataset' and 'src' folders are now ready.


In [2]:
# ==============================================================================
# QUICK FIX: RELOAD THE DATAFRAMES
# ==============================================================================
import pandas as pd
import os

# Define file paths again to be safe
TRAIN_CSV = 'dataset/train.csv'
TEST_CSV = 'dataset/test.csv'

# Load the data into the train_df and test_df variables
print("⏳ Reloading the train and test DataFrames into memory...")
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
print("✅ DataFrames reloaded successfully. You can now proceed!")

⏳ Reloading the train and test DataFrames into memory...
✅ DataFrames reloaded successfully. You can now proceed!


In [3]:
# ==============================================================================
# QUICK RESTORE CELL: Run this if your session disconnects
# ==============================================================================
import pandas as pd
import os
import torch

print("⏳ Restoring session variables...")

# --- Restore Configuration Variables ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_DIR = 'product_images/'
TRAIN_CSV = 'dataset/train.csv'
TEST_CSV = 'dataset/test.csv'
print(f"   - Device set to: {DEVICE}")
print(f"   - Image directory set to: {IMAGE_DIR}")


# --- Restore DataFrames ---
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
print("   - train_df and test_df reloaded.")

print("\n✅ Restore complete! You are ready to proceed.")

⏳ Restoring session variables...
   - Device set to: cuda
   - Image directory set to: product_images/
   - train_df and test_df reloaded.

✅ Restore complete! You are ready to proceed.


In [11]:
# ==============================================================================
# STAGE 2: FEATURE ENGINEERING
# ==============================================================================
import re
import numpy as np
import torch
import timm
from PIL import Image
from torchvision import transforms
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import pandas as pd # Re-importing just in case

# --- Configuration ---
# Set the device to use GPU if available, otherwise CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {DEVICE}")
# ==============================================================================
# NEW FEATURE ENGINEERING (Add this to the start of Stage 2)
# ==============================================================================
print("\n💡 [New Step] Engineering advanced text features...")

# --- Feature 1: Extract Size/Weight/Volume ---
# This looks for patterns like "500ml", "16 oz", "2.5 kg" to capture product size.
def get_measurements(text):
    text = str(text).lower()
    match = re.search(r'(\d+\.?\d*)\s*(ml|l|oz|g|kg|lb|pounds|ounces|grams|kilograms)\b', text)
    if match:
        return float(match.group(1)) # Return the number (e.g., 500)
    return 0 # Return 0 if no measurement is found

train_df['measurement_value'] = train_df['catalog_content'].apply(get_measurements)
test_df['measurement_value'] = test_df['catalog_content'].apply(get_measurements)
print("   - ✅ Created 'measurement_value' feature.")

# --- Feature 2: Extract Brand Keywords ---
# A simple but effective way to signal brand value to the model.
# TODO: Look through your train.csv and add more brand names to this list!
BRANDS = [
    'sony', 'apple', 'samsung', 'nike', 'adidas', 'salerno', 'lego', 'amazon',
    'google', 'microsoft', 'dell', 'hp', 'lenovo', 'logitech', 'bose', 'nespresso'
]
def contains_brand(text):
    text = str(text).lower()
    for brand in BRANDS:
        if brand in text:
            return 1 # Returns 1 if a known brand is found
    return 0 # Returns 0 otherwise

train_df['has_brand'] = train_df['catalog_content'].apply(contains_brand)
test_df['has_brand'] = test_df['catalog_content'].apply(contains_brand)
print("   - ✅ Created 'has_brand' feature.")


# --- Feature 3: Text Length ---
# Longer, more detailed descriptions can sometimes correlate with higher-value items.
train_df['text_length'] = train_df['catalog_content'].str.len().fillna(0)
test_df['text_length'] = test_df['catalog_content'].str.len().fillna(0)
print("   - ✅ Created 'text_length' feature.")

# --- 1. Engineered Feature: Item Pack Quantity (IPQ) ---
print("\n⏳ [Step 1/3] Extracting Item Pack Quantity (IPQ)...")

def get_ipq(text):
    # Ensure text is a string and lowercase for consistent matching
    text = str(text).lower()
    # This regex looks for patterns like "pack of 12", "12-pack", "ipq-12", "count of 12"
    match = re.search(r'(pack of|pack|count of|ipq-|ipq:|of)\s*(\d+)|(\d+)\s*(-pack|-count)', text)
    if match:
        # The number could be in different capturing groups depending on the regex match
        num = match.group(2) or match.group(3)
        if num:
            return int(num)
    # If no pack information is found, we assume it's a single item (IPQ=1).
    return 1

train_df['ipq'] = train_df['catalog_content'].apply(get_ipq)
test_df['ipq'] = test_df['catalog_content'].apply(get_ipq)
print("✅ IPQ feature created successfully.")
print("   Sample IPQ counts from training data:\n", train_df['ipq'].value_counts().head())

# --- 2. Text Features: Sentence Embeddings ---
print("\n⏳ [Step 2/3] Generating text embeddings with 'all-mpnet-base-v2'. This will take 15-30 minutes...")
# Load the powerful sentence transformer model onto the GPU
text_model = SentenceTransformer('all-mpnet-base-v2', device=DEVICE)

# Fill any potential missing text with an empty string to avoid errors
train_texts = train_df['catalog_content'].fillna('').tolist()
test_texts = test_df['catalog_content'].fillna('').tolist()

# Encode the texts into numerical vectors. The progress bar will show the status.
# batch_size is tuned for a T4 GPU to maximize speed.
train_text_embeddings = text_model.encode(train_texts, show_progress_bar=True, batch_size=64)
test_text_embeddings = text_model.encode(test_texts, show_progress_bar=True, batch_size=64)
print("✅ Text embeddings generated. Shape:", train_text_embeddings.shape)


# --- 3. Image Features: Vision Embeddings ---
print("\n⏳ [Step 3/3] Generating image embeddings with 'EfficientNet-B0'. This may take 20-40 minutes...")
# Load a pre-trained image model (EfficientNet-B0 is a great balance of speed and accuracy)
img_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0).to(DEVICE)
img_model.eval() # Set the model to evaluation mode

# Get the specific image transformations (resize, normalize, etc.) required by this model
data_config = timm.data.resolve_model_data_config(img_model)
transform = timm.data.create_transform(**data_config, is_training=False)

# This function processes one image link at a time, making it robust to errors
def get_image_embedding(image_link, folder):
    EMBEDDING_DIM = 1280 # efficientnet_b0 outputs a vector of 1280 numbers

    # Handle missing or invalid links gracefully by returning a vector of zeros
    if pd.isna(image_link):
        return np.zeros(EMBEDDING_DIM)

    try:
        # Construct the local path to the downloaded image
        filename = image_link.split('/')[-1]
        image_path = os.path.join(folder, filename)

        # If the image wasn't downloaded for some reason, return zeros
        if not os.path.exists(image_path):
             return np.zeros(EMBEDDING_DIM)

        # Open image, apply transforms, and get the embedding vector from the model
        img = Image.open(image_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(DEVICE) # Add batch dimension and send to GPU
        with torch.no_grad(): # Disable gradient calculation for efficiency
            embedding = img_model(img_tensor).cpu().numpy().flatten()
        return embedding
    except Exception as e:
        # If anything else goes wrong (e.g., corrupt image file), return a zero vector
        return np.zeros(EMBEDDING_DIM)

# Loop through all image links and generate embeddings. The tqdm bar will show progress.
train_image_embeddings = np.array([get_image_embedding(link, IMAGE_DIR) for link in tqdm(train_df['image_link'], desc="Processing train images")])
test_image_embeddings = np.array([get_image_embedding(link, IMAGE_DIR) for link in tqdm(test_df['image_link'], desc="Processing test images")])
print("✅ Image embeddings generated. Shape:", train_image_embeddings.shape)
print("\n✅🎉 Stage 2 Complete: All features have been engineered!")

✅ Using device: cuda

💡 [New Step] Engineering advanced text features...
   - ✅ Created 'measurement_value' feature.
   - ✅ Created 'has_brand' feature.
   - ✅ Created 'text_length' feature.

⏳ [Step 1/3] Extracting Item Pack Quantity (IPQ)...
✅ IPQ feature created successfully.
   Sample IPQ counts from training data:
 ipq
1     51808
6      4898
12     4296
2      2689
3      2369
Name: count, dtype: int64

⏳ [Step 2/3] Generating text embeddings with 'all-mpnet-base-v2'. This will take 15-30 minutes...


Batches:   0%|          | 0/1172 [00:00<?, ?it/s]

Batches:   0%|          | 0/1172 [00:00<?, ?it/s]

✅ Text embeddings generated. Shape: (75000, 768)

⏳ [Step 3/3] Generating image embeddings with 'EfficientNet-B0'. This may take 20-40 minutes...


Processing train images:   0%|          | 0/75000 [00:00<?, ?it/s]

Processing test images:   0%|          | 0/75000 [00:00<?, ?it/s]

✅ Image embeddings generated. Shape: (75000, 1280)

✅🎉 Stage 2 Complete: All features have been engineered!


In [5]:
# ==============================================================================
# CHECKPOINT - Step 1: Mount Google Drive
# ==============================================================================
from google.colab import drive
import os

print("⏳ Connecting to your Google Drive...")
# This command will prompt you for authorization.
drive.mount('/content/drive')
print("✅ Google Drive connected successfully.")

# We will create a dedicated folder in your Drive to keep things organized.
SAVE_PATH = '/content/drive/MyDrive/ML_Challenge_Features'
os.makedirs(SAVE_PATH, exist_ok=True)
print(f"   - Files will be saved in: {SAVE_PATH}")

⏳ Connecting to your Google Drive...
Mounted at /content/drive
✅ Google Drive connected successfully.
   - Files will be saved in: /content/drive/MyDrive/ML_Challenge_Features


In [6]:
# ==============================================================================
# CHECKPOINT - Step 2: Save Features
# ==============================================================================
import numpy as np # Make sure numpy is imported

print(f"\n⏳ Saving all engineered features to your Drive. This may take a minute...")

# We use np.savez_compressed to efficiently save all the large embedding arrays
# into a single, compact file named 'engineered_features.npz'.
np.savez_compressed(
    os.path.join(SAVE_PATH, 'engineered_features.npz'),
    train_text=train_text_embeddings,
    test_text=test_text_embeddings,
    train_image=train_image_embeddings,
    test_image=test_image_embeddings
)

# We also save the dataframes as CSVs to preserve the new 'ipq' column.
train_df.to_csv(os.path.join(SAVE_PATH, 'train_with_ipq.csv'), index=False)
test_df.to_csv(os.path.join(SAVE_PATH, 'test_with_ipq.csv'), index=False)

print("\n✅🎉 Checkpoint Complete! All your hard work is now safely stored in Google Drive.")
print("You can now proceed with model training.")


⏳ Saving all engineered features to your Drive. This may take a minute...

✅🎉 Checkpoint Complete! All your hard work is now safely stored in Google Drive.
You can now proceed with model training.


In [12]:
# ==============================================================================
# STAGE 3: FEATURE FUSION & TARGET TRANSFORMATION
# ==============================================================================
import lightgbm as lgb
import xgboost as xgb
import numpy as np # Ensure numpy is imported

print("✅ Checkpoint complete. Starting the final stages...")
print("\n⏳ [Stage 3/5] Combining all features into final training matrices...")

# Combine the features horizontally: (text embeddings, image embeddings, ipq)
# We need to reshape 'ipq' to be a single-column 2D array for stacking.
# REPLACE the old hstack section in Stage 3 with this new one
# ==============================================================================

# Combine all features: embeddings + all our new engineered columns
X_train = np.hstack([
    train_text_embeddings,
    train_image_embeddings,
    train_df['ipq'].values.reshape(-1, 1),
    train_df['measurement_value'].values.reshape(-1, 1),
    train_df['has_brand'].values.reshape(-1, 1),
    train_df['text_length'].values.reshape(-1, 1)
])

X_test = np.hstack([
    test_text_embeddings,
    test_image_embeddings,
    test_df['ipq'].values.reshape(-1, 1),
    test_df['measurement_value'].values.reshape(-1, 1),
    test_df['has_brand'].values.reshape(-1, 1),
    test_df['text_length'].values.reshape(-1, 1)
])

# --- Target Transformation for Low SMAPE ---
# Predicting the logarithm of the price is the key to performing well on relative error metrics like SMAPE.
# We use np.log1p which calculates log(1 + x) to robustly handle prices of 0.
y_train_log = np.log1p(train_df['price'])

print("✅ Features combined successfully.")
print(f"   - Final training feature shape: {X_train.shape}")
print(f"   - Final test feature shape:     {X_test.shape}")


# ==============================================================================
# STAGE 4: ENSEMBLE MODELING 🧠
# ==============================================================================
print("\n⏳ [Stage 4/5] Training the ensemble models...")

# --- LightGBM Model ---
# These parameters are a strong starting point, balancing speed and accuracy.
lgbm_params = {
    'objective': 'regression_l1',  # MAE is robust to price outliers
    'metric': 'mae',
    'n_estimators': 2000,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'num_leaves': 40, # Increased slightly for more complexity
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}

print("   - Training LightGBM model...")
lgbm_model = lgb.LGBMRegressor(**lgbm_params)
lgbm_model.fit(X_train, y_train_log)
print("   - LightGBM training complete.")

# --- XGBoost Model ---
# XGBoost attacks the problem differently, making it a perfect ensemble partner.
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'n_estimators': 1500,
    'learning_rate': 0.015,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'seed': 42,
    'n_jobs': -1,
    'tree_method': 'hist', # Use the fast histogram-based method
    'device': 'cuda',      # Explicitly tell XGBoost to use the GPU
}
print("   - Training XGBoost model...")
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train_log)
print("   - XGBoost training complete.")

print("✅ Ensemble models trained successfully.")


# ==============================================================================
# STAGE 5: PREDICTION & SUBMISSION
# ==============================================================================
print("\n⏳ [Stage 5/5] Generating final predictions and submission file...")

# Predict the log-price from both models
lgbm_log_preds = lgbm_model.predict(X_test)
xgb_log_preds = xgb_model.predict(X_test)

# --- Ensemble Averaging ---
# The core of the ensemble: average the predictions to get a more robust result.
avg_log_preds = (lgbm_log_preds + xgb_log_preds) / 2.0

# --- Inverse Transformation ---
# We must convert our predictions back from the log scale to the actual price scale.
# np.expm1 calculates exp(x) - 1, which is the exact inverse of np.log1p.
final_predictions = np.expm1(avg_log_preds)

# As per the competition rules, ensure all predicted prices are positive.
final_predictions[final_predictions < 0] = 0

# Create the submission DataFrame in the required format.
submission_df = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': final_predictions
})

# Save the final file to your Colab session.
submission_df.to_csv('submission.csv', index=False)

print("\n\n✅🎉 ALL STAGES COMPLETE! ✅🎉")
print("Your final 'submission.csv' file is ready for download in the file explorer.")

✅ Checkpoint complete. Starting the final stages...

⏳ [Stage 3/5] Combining all features into final training matrices...
✅ Features combined successfully.
   - Final training feature shape: (75000, 2052)
   - Final test feature shape:     (75000, 2052)

⏳ [Stage 4/5] Training the ensemble models...
   - Training LightGBM model...
   - LightGBM training complete.
   - Training XGBoost model...
   - XGBoost training complete.
✅ Ensemble models trained successfully.

⏳ [Stage 5/5] Generating final predictions and submission file...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(




✅🎉 ALL STAGES COMPLETE! ✅🎉
Your final 'submission.csv' file is ready for download in the file explorer.


In [10]:
# ==============================================================================
# EVALUATION STAGE: Measure Performance on a Validation Set
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np # Ensure numpy is imported
import pandas as pd # Ensure pandas is imported

# --- 1. Define the SMAPE Metric Function ---
# This function calculates the competition's specific evaluation metric.
def calculate_smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle cases where both true and pred are zero to avoid division by zero
    # np.where(condition, value_if_true, value_if_false)
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 2. Split Your Data ---
# We'll use 80% of the data for training and hold back 20% for validation.
# We split the feature matrices (X_train) and the target vector (y_train_log).
X_train_split, X_val, y_train_split, y_val_log = train_test_split(
    X_train, y_train_log, test_size=0.2, random_state=42
)
print(f"Data split for validation: {X_train_split.shape[0]} training samples, {X_val.shape[0]} validation samples.")


# --- 3. Train a Model on the Split Data ---
# We will train a LightGBM model just on the 80% training portion.
print("\n⏳ Training a temporary model for evaluation...")
eval_model = lgb.LGBMRegressor(**lgbm_params) # Using the same params as your final model
eval_model.fit(X_train_split, y_train_split)


# --- 4. Make Predictions on the Validation Set ---
print("✅ Model trained. Making predictions on the unseen validation data...")
val_preds_log = eval_model.predict(X_val)

# IMPORTANT: Inverse transform both predictions and actual values back to the original price scale
val_preds_price = np.expm1(val_preds_log)
y_val_price = np.expm1(y_val_log)

# Ensure prices are positive
val_preds_price[val_preds_price < 0] = 0


# --- 5. Calculate and Display Performance Metrics ---
print("\n--- Model Performance Evaluation ---")

# SMAPE Score (The most important metric for the competition)
smape_score = calculate_smape(y_val_price, val_preds_price)
print(f"🏆 SMAPE Score: {smape_score:.4f}%")
print("   (Lower is better. This is your estimated leaderboard score.)")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_val_price, val_preds_price)
print(f"\n💡 Mean Absolute Error (MAE): ${mae:.2f}")
print("   (On average, the model's price prediction is off by this amount.)")

# R-squared (R²)
r2 = r2_score(y_val_price, val_preds_price)
print(f"\n💡 R-squared (R²): {r2:.4f}")
print("   (Represents the percentage of price variation the model can explain. Closer to 1 is better.)")
print("\n------------------------------------")

Data split for validation: 60000 training samples, 15000 validation samples.

⏳ Training a temporary model for evaluation...
✅ Model trained. Making predictions on the unseen validation data...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



--- Model Performance Evaluation ---
🏆 SMAPE Score: 59.5891%
   (Lower is better. This is your estimated leaderboard score.)

💡 Mean Absolute Error (MAE): $13.98
   (On average, the model's price prediction is off by this amount.)

💡 R-squared (R²): 0.0880
   (Represents the percentage of price variation the model can explain. Closer to 1 is better.)

------------------------------------


In [9]:
# ==============================================================================
# FINAL SAVE: Save Trained Models and Submission File
# ==============================================================================
import joblib
import os
from google.colab import drive

# --- 1. Mount Google Drive (if not already mounted) ---
# If it's already mounted, this will run very quickly.
if not os.path.isdir('/content/drive/MyDrive'):
    print("⏳ Connecting to your Google Drive...")
    drive.mount('/content/drive')
    print("✅ Google Drive connected.")

# --- 2. Define the same save path you used before ---
SAVE_PATH = '/content/drive/MyDrive/ML_Challenge_Features'
print(f"Files will be saved in: {SAVE_PATH}")


# --- 3. Save the Trained Models ---
# We use joblib, which is excellent for saving ML models.
print("\n⏳ Saving the trained LightGBM and XGBoost models...")

# Save the LightGBM model
joblib.dump(lgbm_model, os.path.join(SAVE_PATH, 'lgbm_model.pkl'))

# Save the XGBoost model
joblib.dump(xgb_model, os.path.join(SAVE_PATH, 'xgb_model.pkl'))

print("✅ Models saved successfully to your Drive.")


# --- 4. Save a Backup of Your Submission File ---
print("\n⏳ Saving a backup of your submission.csv...")
submission_df.to_csv(os.path.join(SAVE_PATH, 'submission_backup.csv'), index=False)
print("✅ Submission file backed up successfully.")

print("\n\n✅🎉 Project Fully Saved! You can now close this notebook and restore your work anytime.")

Files will be saved in: /content/drive/MyDrive/ML_Challenge_Features

⏳ Saving the trained LightGBM and XGBoost models...
✅ Models saved successfully to your Drive.

⏳ Saving a backup of your submission.csv...
✅ Submission file backed up successfully.


✅🎉 Project Fully Saved! You can now close this notebook and restore your work anytime.


In [13]:
# ==============================================================================
# SAVE PROGRESS: Version 2 ("Draft 2")
# ==============================================================================
import joblib
import os
from google.colab import drive

# --- 1. Mount Google Drive (if not already connected) ---
if not os.path.isdir('/content/drive/MyDrive'):
    print("⏳ Connecting to your Google Drive...")
    drive.mount('/content/drive')
    print("✅ Google Drive connected.")

# --- 2. Define the save path (same folder as before) ---
SAVE_PATH = '/content/drive/MyDrive/ML_Challenge_Features'
print(f"Files will be saved in: {SAVE_PATH}")


# --- 3. Save the new, improved models with a "_draft2" suffix ---
print("\n⏳ Saving the new 'Draft 2' models...")

# Save the LightGBM model
joblib.dump(lgbm_model, os.path.join(SAVE_PATH, 'lgbm_model_draft2.pkl'))

# Save the XGBoost model
joblib.dump(xgb_model, os.path.join(SAVE_PATH, 'xgb_model_draft2.pkl'))

print("✅ 'Draft 2' models saved successfully to your Drive.")


# --- 4. Save a backup of your new submission file ---
# This links your saved models to the submission they produced.
print("\n⏳ Saving a backup of your new submission_draft2.csv...")
# Assuming the submission DataFrame is named submission_df
submission_df.to_csv(os.path.join(SAVE_PATH, 'submission_draft2.csv'), index=False)
print("✅ New submission file backed up successfully.")

print("\n\n✅🎉 Project 'Draft 2' Fully Saved!")

Files will be saved in: /content/drive/MyDrive/ML_Challenge_Features

⏳ Saving the new 'Draft 2' models...
✅ 'Draft 2' models saved successfully to your Drive.

⏳ Saving a backup of your new submission_draft2.csv...
✅ New submission file backed up successfully.


✅🎉 Project 'Draft 2' Fully Saved!


In [14]:
# ==============================================================================
# EVALUATION STAGE (for the new Draft 2 model)
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np # Ensure numpy is imported
import pandas as pd # Ensure pandas is imported

# --- 1. Define the SMAPE Metric Function ---
def calculate_smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 2. Split Your Data (using the new, wider X_train) ---
X_train_split, X_val, y_train_split, y_val_log = train_test_split(
    X_train, y_train_log, test_size=0.2, random_state=42
)
print(f"Data split for validation: {X_train_split.shape[0]} training samples, {X_val.shape[0]} validation samples.")


# --- 3. Train a Model on the Split Data ---
print("\n⏳ Training a temporary model for evaluation...")
# Use the same lgbm_params you used for your final model for an accurate comparison
eval_model = lgb.LGBMRegressor(**lgbm_params)
eval_model.fit(X_train_split, y_train_split)


# --- 4. Make Predictions on the Validation Set ---
print("✅ Model trained. Making predictions on the unseen validation data...")
val_preds_log = eval_model.predict(X_val)

# Inverse transform both predictions and actual values back to the price scale
val_preds_price = np.expm1(val_preds_log)
y_val_price = np.expm1(y_val_log)
val_preds_price[val_preds_price < 0] = 0


# --- 5. Calculate and Display Performance Metrics ---
print("\n--- NEW Model Performance (Draft 2) ---")

# SMAPE Score (The most important metric)
smape_score = calculate_smape(y_val_price, val_preds_price)
print(f"🏆 SMAPE Score: {smape_score:.4f}%")
print("   (Compare this to your previous score of ~59.59%)")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_val_price, val_preds_price)
print(f"\n💡 Mean Absolute Error (MAE): ${mae:.2f}")

# R-squared (R²)
r2 = r2_score(y_val_price, val_preds_price)
print(f"\n💡 R-squared (R²): {r2:.4f}")
print("   (Compare this to your previous score of ~0.0880. Higher is better!)")
print("\n------------------------------------")

Data split for validation: 60000 training samples, 15000 validation samples.

⏳ Training a temporary model for evaluation...
✅ Model trained. Making predictions on the unseen validation data...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



--- NEW Model Performance (Draft 2) ---
🏆 SMAPE Score: 59.1005%
   (Compare this to your previous score of ~59.59%)

💡 Mean Absolute Error (MAE): $13.88

💡 R-squared (R²): 0.0900
   (Compare this to your previous score of ~0.0880. Higher is better!)

------------------------------------


In [15]:
# ==============================================================================
# STRATEGY 1: ERROR ANALYSIS (CRITICAL NEXT STEP)
# ==============================================================================

print("🕵️‍♀️ Analyzing the model's biggest mistakes...")

# Create a DataFrame to hold the results from your validation split
error_df = pd.DataFrame({
    'catalog_content': train_df.loc[y_val_log.index]['catalog_content'],
    'actual_price': y_val_price,
    'predicted_price': val_preds_price,
})

# Calculate the SMAPE for each individual prediction
numerator = np.abs(error_df['predicted_price'] - error_df['actual_price'])
denominator = (np.abs(error_df['actual_price']) + np.abs(error_df['predicted_price'])) / 2
error_df['individual_smape'] = np.where(denominator == 0, 0, numerator / denominator) * 100


# --- Display the Worst Predictions ---
print("\n--- Top 20 Worst Predictions (by SMAPE) ---")
# Using .options.display so you can see the full text content
pd.options.display.max_colwidth = 200
print(error_df.sort_values('individual_smape', ascending=False).head(20))

🕵️‍♀️ Analyzing the model's biggest mistakes...

--- Top 20 Worst Predictions (by SMAPE) ---
                                                                                                                                                                                               catalog_content  \
58617  Item Name: 4Patriots 1-Year Survival Food Kit: Emergency Food Supply Designed to Last 25 Years, 2,752 Servings\nBullet Point 1: 2,752 EMERGENCY FOOD SERVINGS: 24 delicious survival food recipes li...   
59934       Item Name: Trolli Sour Brite Crawlers Candy, 2 Ounce -- 144 per case.\nBullet Point 1: Sour Brite Crawlers Candy 2 Ounce\nBullet Point 2: Food Items cannot be return\nValue: 144.0\nUnit: Count\n   
18709                                                                                             Item Name: Nescaf Clasico Decaf Instant Coffee 7 Oz\nProduct Description: Grocery\nValue: 7.0\nUnit: Ounce\n   
38615  Item Name: Land O Lakes Arctic White Hot Cocoa Classics Mix,